In [1]:
import pandas as pd
import re

In [2]:
messages  =["""
com.systemone.lc2.manualloadplanning.dto.LoadDTO [ id = 2978137  flightId = 667683 legId = 687494  deleted = false  fragmentId = LoadFragment ]
BULK
 Item Id   Number       Status    Weight         Pieces    Volume    Origin    Onl.Sta.  Dest.     Category  Position  Confirmed  Remarks  
 37125183  NULL         ESTIMATE  26.00     KG   2         0.00      DUB       DUB       CDG       BT        NULL      false     NULL 
 37125182  NULL         ESTIMATE  104.00    KG   8         0.00      DUB       DUB       CDG       B         NULL      false     NULL 
ULD
 Item Id   ULD       Number    Airline   Special   Status    Weight         Pieces    Volume    Origin    Onl.Sta.  Dest.     Category  Position  Confirmed  Remarks  
 37144700  AKH       44778     BA        NULL      ACTUAL    303.00    KG   0         0.00      DUB       DUB       CDG       M         NULL      false     NULL 
 37125180  AKH       NULL      MN                  ESTIMATE  455.00    KG   35        0.00      DUB       DUB       CDG       B         32        false     NULL 
 37144702  AKH       46046     BA        NULL      ACTUAL    94.00     KG   0         0.00      DUB       DUB       CDG       C         NULL      false     NULL 
 37125181  AKH       NULL      MN                  ESTIMATE  455.00    KG   35        0.00      DUB       DUB       CDG       B         41        false     NULL 
 37125179  AKH       NULL      MN                  ESTIMATE  455.00    KG   35        0.00      DUB       DUB       CDG       B         42        false     NULL 
 37144701  AKH       67400     MN        NULL      ACTUAL    333.00    KG   0         0.00      DUB       DUB       CDG       M         NULL      false     NULL 

STATUS FUEL 10 AIRCRAFT_CONFIG 1 EZFW 9 CARGO_FINAL 1 CARGO_TRANSFER 1 OFP 4 CABIN_CONFIG 1 AUTO_MODE_ACTIVE 1 AUTOMATION_STARTED 0 BAG_LOAD_ITEMS_GEN 1 EZFW_COUNTER 9 REGISTRATION 1 REGISTRATION_CHANGE 1 FUEL_ORDER 2
"""]

In [3]:
def metaFunction(messages):
    data = []
    for message in messages:

        uld = findType("ULD", message)
        bulk = findType("BULK", message)
        bulk = list(set(bulk) - set(uld))

        bulk = [line for line in bulk if "STATUS FUEL" not in line]
        bulk = [line for line in bulk if line != "BULK" and line != "ULD" and line != ""]
        columnIndex = bulk.index([line for line in bulk if line.strip().startswith("Item Id")][0])
        bulk[0], bulk[columnIndex] = bulk[columnIndex], bulk[0]

        uldIndices = findStartingCharactersIndices("ULD")
        bulkIndices = findStartingCharactersIndices("BULK")
        
        uldData = fetchData(uld, uldIndices)
        bulkData = fetchData(bulk, bulkIndices)
        combined = uldData + bulkData
        data.append(combined)
    return pd.DataFrame(data[0])

In [4]:
def findType(type, string):
    match = re.search(fr"{type}", string)
    substring = ""
    if match:
        substring += string[match.start():]
    substring = substring.split("\n")
    if type == "ULD":
        return substring[1:-3]
    return substring

def findStartingCharactersIndices(type):
    dummy_row = None
    if type == "ULD":
        dummy_row = "37137862  AKH       NULL      MN        NULL      ESTIMATE  88.00     KG   0         0.00      DUB       DUB       EDI       X         11        false     NULL"
    else:
        dummy_row = "37144805  NULL         ESTIMATE  0.00      KG   0         0.00      DUB       DUB       EDI       B         5         false     NULL"
    dummy_row = dummy_row.replace("KG", "")
    words = dummy_row.split()
    indices = []
    current_index = 0

    # Iterate through each word
    for word in words:
        # Get the first character of the word
        first_char = word[0]
        # Find the index of the first character in the original string
        index = dummy_row.index(first_char, current_index)
        # Append the index and character to the lists
        indices.append(index)
        # Update the current index to search for the next word
        current_index = index + len(word)
    return indices

def fetchData(substring, indices):
    columns = re.sub(r'\s+', ',', substring[0].lstrip().replace("Item Id", "ItemID"))[:-1].split(",")
    data = []
    for line in substring[1:]:
        line = line.lstrip()
        line = line.replace("KG","")
        dummy_row_list = list(line)
        for index in indices:
            if dummy_row_list[index] == " ":
                # Replace the character and the following 3 characters with "NULL"
                dummy_row_list[index:index+4] = "NULL"

        # Convert the list back to a string
        line = ''.join(dummy_row_list)

        line = re.sub(r'\s+', ',', line)
        line = line.split(",")
        dictionary = dict(zip(columns, line))
        data.append(dictionary)
    return data

In [5]:
df = metaFunction(messages)
df

,ItemID,ULD,Number,Airline,Special,Status,Weight,Pieces,Volume,Origin,Onl.Sta.,Dest.,Category,Position,Confirmed,Remarks
0,37144700,AKH,44778,BA,NULL,ACTUAL,303.00,0,0.00,DUB,DUB,CDG,M,NULL,false,NULL
1,37125180,AKH,NULL,MN,NULL,ESTIMATE,455.00,35,0.00,DUB,DUB,CDG,B,32,false,NULL
2,37144702,AKH,46046,BA,NULL,ACTUAL,94.00,0,0.00,DUB,DUB,CDG,C,NULL,false,NULL
3,37125181,AKH,NULL,MN,NULL,ESTIMATE,455.00,35,0.00,DUB,DUB,CDG,B,41,false,NULL
4,37125179,AKH,NULL,MN,NULL,ESTIMATE,455.00,35,0.00,DUB,DUB,CDG,B,42,false,NULL
5,37144701,AKH,67400,MN,NULL,ACTUAL,333.00,0,0.00,DUB,DUB,CDG,M,NULL,false,NULL
6,37125183,NaN,NULL,NaN,NaN,ESTIMATE,26.00,2,0.00,DUB,DUB,CDG,BT,NULL,false,NULL
7,37125182,NaN,NULL,NaN,NaN,ESTIMATE,104.00,8,0.00,DUB,DUB,CDG,B,NULL,false,NULL
